In [1]:
from speechbrain.inference.interfaces import foreign_class
import pandas as pd
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix

/home/paige/miniconda3/envs/speechbrain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
classifier = foreign_class(
    source="/home/paige/Documents/BERST/speech_brain/results_2/train_with_wav2vec2/1993/save/CKPT+2024-07-11+20-47-22+00",
    pymodule_file="/home/paige/Documents/BERST/speech_brain/recipes/IEMOCAP/emotion_recognition/custom_interface.py",
    hparams_file="/home/paige/Documents/BERST/speech_brain/recipes/IEMOCAP/emotion_recognition/hparams/inference_7.yaml",
    classname="CustomEncoderWav2vec2Classifier",
    savedir="/home/paige/Documents/BERST/speech_brain/results_2/train_with_wav2vec2/1993/save/CKPT+2024-07-11+20-47-22+00"
)

/home/paige/miniconda3/envs/speechbrain/lib/python3.12/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
speechbrain.lobes.models.huggingface_transformers.huggingface - Wav2Vec2Model is frozen.
/home/paige/miniconda3/envs/speechbrain/lib/python3.12/site-packages/speechbrain/utils/checkpoints.py:152: Fu

In [3]:
test_data = pd.read_csv("/home/paige/Documents/BERST/speech_brain/recipes/IEMOCAP/emotion_recognition/IEMOCAP/session_2_speechbrain.csv")
path = "/home/paige/Documents/BERST/"

out_probs = []
scores = []
indexes = []
text_labs = []
for index, file in test_data.iterrows():
    out_prob, score, index, text_lab = classifier.classify_file(f"{path}/{file['wav']}")
    out_probs.append(out_prob)
    scores.append(score)
    indexes.append(index)
    text_labs.append(text_lab)

In [4]:
predictions = [label for file in text_labs for label in file]
predictions = [file.replace('[Neutral]', 'Neutral') for file in predictions]
predictions = [file.replace('[Sadness]', 'Sadness') for file in predictions]
predictions = [file.replace('[Happiness]', 'Happiness') for file in predictions]
predictions = [file.replace('[Anger]', 'Anger') for file in predictions]
predictions = [file.replace('[Fear]', 'Fear') for file in predictions]
predictions = [file.replace('[Surprise]', 'Surprise') for file in predictions]
predictions = [file.replace('[Disgust]', 'Disgust') for file in predictions]

In [5]:
test_data['predictions'] = predictions

In [6]:
test_data.to_csv('iemocap_7_class_session_2.csv', index=False)

In [7]:
print(f"UA score: {accuracy_score(test_data['affect'], test_data['predictions'])}")
print(f"WA score: {balanced_accuracy_score(test_data['affect'], test_data['predictions'])}")

UA score: 0.0673076923076923
WA score: 0.053242780480925525


In [45]:
test_data

,wav,affect,length,predictions
0,IEMOCAP_full_release/Session4/sentences/wav/Se...,Anger,2.581437,Anger
1,IEMOCAP_full_release/Session4/sentences/wav/Se...,Anger,3.041937,Anger
2,IEMOCAP_full_release/Session4/sentences/wav/Se...,Anger,3.990000,Anger
3,IEMOCAP_full_release/Session4/sentences/wav/Se...,Anger,3.219937,Anger
4,IEMOCAP_full_release/Session4/sentences/wav/Se...,Anger,3.090375,Anger
...,...,...,...,...
3384,IEMOCAP_full_release/Session4/sentences/wav/Se...,Neutral,3.870000,Neutral
3385,IEMOCAP_full_release/Session4/sentences/wav/Se...,Neutral,2.779937,Neutral
3386,IEMOCAP_full_release/Session4/sentences/wav/Se...,Neutral,9.955312,Neutral
3387,IEMOCAP_full_release/Session4/sentences/wav/Se...,Neutral,2.340000,Neutral


In [44]:
cf_matrix = confusion_matrix(test_data['predictions'], test_data['affect'])
print(cf_matrix)

[[986   3   5   2  18  17   8]
 [  1   6   0   2   2   1   0]
 [  0   0  22   0   0   0   1]
 [  1   0   0 314  59   3   0]
 [ 76   0   6 179 999  36  19]
 [ 13   0  11   8  69 431   1]
 [  3   0   2   3  10   0  72]]
